# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

# **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [2]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3827, 3831, 3832]


In [3]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3827,
    "name": "Luna",
    "account": {
      "id": 4062,
      "number": "02023-7",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 3719,
      "number": "**** **** **** 0423",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 8078,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "O Santander tem solu\u00e7\u00f5es de cr\u00e9dito sob medida pra voc\u00ea. Confira!"
      },
      {
        "id": 8079,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Invista hoje para um futuro pr\u00f3spero! #InvistaComSabedoria"
      }
    ]
  },
  {
    "id": 3831,
    "name": "Matheus",
    "account": {
      "id": 4066,
      "number": "02023-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
     

# **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [4]:
##!pip install openai

In [5]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
# openai_api_key = 'MinhaAPi'

Como o GPT Atualmente está cobrando mesmo com o free trial decidi criar minhas proprias mensagens

##GPT

In [6]:
# import openai

# openai.api_key = openai_api_key

# def generate_ai_news(user):
#     completion = openai.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#       {
#           "role": "system",
#           "content": "Você é um especialista em marketing bancário."
#       },
#       {
#           "role": "user",
#           "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
#       }
#     ]
#     )
#     return completion.choices[0].message.content.strip('\"')

# for user in users:
#   news = generate_ai_news(user)
#   print(news)
#   user['news'].append({"icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
#       "description": news})



##Estático

In [7]:
import random

def generate_generic_news(user):
    # Lista de mensagens genéricas para diferentes cenários
    messages = [
        f"{user['name']}, Invista hoje e garanta um futuro mais tranquilo.",
        f"{user['name']}, Diversifique seus investimentos para reduzir riscos.",
        f"{user['name']}, Faça seu dinheiro trabalhar por você!",
        f"{user['name']}, Comece com pequenos investimentos e veja os resultados crescerem.",
         f"{user['name']}, Planeje seu futuro com inteligência financeira.",
    ]

    # Escolhe uma mensagem aleatória para cada usuário
    return random.choice(messages)

for user in users:
    news = generate_generic_news(user)
    print(news)
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })

Luna, Faça seu dinheiro trabalhar por você!
Matheus, Invista hoje e garanta um futuro mais tranquilo.
Leni, Comece com pequenos investimentos e veja os resultados crescerem.


#Load

In [10]:
import json
local_json_file = "users.json"

def load_users():
    try:
        with open(local_json_file, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return []

def save_users(users):
    with open(local_json_file, "w") as file:
        json.dump(users, file, indent=4)

def update_user(user):
    users = load_users()
    for idx, existing_user in enumerate(users):
        if existing_user["id"] == user["id"]:
            users[idx] = user
            save_users(users)
            return True
    return False


save_users(users)
users_local = load_users()

In [9]:
for user in users_local:
    user["last_updated"] = "2025-01-26"

    success = update_user(user)
    print(f"User {user['name']} updated? {success}")

User Luna updated? True
User Matheus updated? True
User Leni updated? True


Modificações : Tentei criar um jason local para facilitar o carregamento e teste de json antes de colocar o codigo em produção. Sendo assim, pode ser criado um cenário mais controlado para devs menos experientes.
Vantagens

    

*  Simples: Usa um arquivo JSON para simular o comportamento de uma API.
*  Flexível: Fácil de modificar e expandir para novas funcionalidades.